Setup PyTorch to use best hardware option

In [ ]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

Get model and tokenizer from Hugginface

In [2]:
from transformers import DebertaV2Tokenizer, DebertaV2Model

model_name = "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name, clean_up_tokenization_spaces=False, use_fast=True)
model = DebertaV2Model.from_pretrained(model_name).to(device)
model.eval()

Dataset load

In [5]:
ARTIFACTS_BASE = '../../../artifacts'

In [ ]:
from os import path
from datasets import load_from_disk

dataset_path = path.join(ARTIFACTS_BASE, 'step-1-classic-ml', 'deberta-v3-base')

train_dataset = load_from_disk(path.join(dataset_path, 'train'))
test_dataset = load_from_disk(path.join(dataset_path, 'test'))

Prepare loaded dataset for metrics counting

In [9]:
import numpy as np

X_test_embedding_list = np.array(test_dataset["embedding"])
y_test_label_list = np.array(test_dataset["label"])

X_train_embedding_list = np.array(train_dataset["embedding"])
y_train_label_list = np.array(train_dataset["label"])

In [ ]:
# Check number of training and testing samples
print(f"#Training Samples: {len(X_train_embedding_list)}")
print(f"#Testing Samples: {len(X_test_embedding_list)}")

In [22]:
# Import classification models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier

# Initialize estimators using their default parameters
estimators = [
    ("Naive Bayes", GaussianNB()),
    ("LGB Classifier", lgb.LGBMClassifier()),
    ("Random Forest", RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1)),
    ("MLPClassifier", MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42)),
]

In [ ]:
# Import performance metrics libraries
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# Prepare a DataFrame to keep track of the models' performance
results = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1 score"])

# Iterate through each estimator in the list
for est_name, est_obj in tqdm(desc='Calculate metrics', iterable=estimators):

    if hasattr(est_obj, 'to'):
        est_obj.to(device)  # Move model to GPU if supported
    
    # Fit the model
    est_obj.fit(X_train_embedding_list, y_train_label_list)
    
    # Use the model to predict unseen prompts
    y_predict = est_obj.predict(X_test_embedding_list)
    
    # Calculate performance metrics
    accuracy = accuracy_score(y_test_label_list, y_predict)
    precision = precision_score(y_test_label_list, y_predict)
    recall = recall_score(y_test_label_list, y_predict)
    f1 = f1_score(y_test_label_list, y_predict)  
    
    # Store performance metrics
    results.loc[est_name] = [accuracy, precision, recall, f1]

In [ ]:
results